In [3]:
import pandas as pd

In [13]:
def format_data(filename):
    df = pd.read_csv(filename)

    entire_sup_rent = df[
        (df['Lloguer_mitja'] == 'Lloguer mitjÃ  mensual (Euros/mes)') | 
        (df['Lloguer_mitja'] == 'Lloguer mitjà mensual (Euros/mes)')
    ]['Preu']
    
    m2_rent = df[
        (df['Lloguer_mitja'] == 'Lloguer mitjÃ  per superfÃ­cie (Euros/m2 mes)') | 
        (df['Lloguer_mitja'] == 'Lloguer mitjà per superfície (Euros/m2 mes)')
    ]['Preu']
    

    df = df.drop(df[
        (df['Lloguer_mitja'] == 'Lloguer mitjÃ  per superfÃ­cie (Euros/m2 mes)') |
        (df['Lloguer_mitja'] == 'Lloguer mitjà per superfície (Euros/m2 mes)')
    ].index)

    df = df.drop('Lloguer_mitja', axis=1)
    df = df.drop('Preu', axis=1)


    df['Preu'] = entire_sup_rent.values
    df['PreuM2'] = m2_rent.values
    df.rename(columns = {'Any':'Year'}, inplace = True)

    return df

In [39]:
files = ['2014.csv', '2015.csv', '2016.csv', '2017.csv', '2018.csv', '2019.csv', '2020.csv', '2021.csv']

res = pd.DataFrame()
for file in files:
    df = format_data(file)
    res = pd.concat([res, df])
    
res['id'] = [i for i in range(len(res))]
res

,Year,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Preu,PreuM2,id
0,2014,1,1,Ciutat Vella,1,el Raval,589.55,10.76,0
1,2014,1,1,Ciutat Vella,2,el Barri GÃ²tic,712.79,10.58,1
2,2014,1,1,Ciutat Vella,3,la Barceloneta,540.71,14.4,2
3,2014,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",673.44,11.01,3
4,2014,1,2,Eixample,5,el Fort Pienc,736.09,10.42,4
...,...,...,...,...,...,...,...,...,...
506,2021,4,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,1205.3,15.2,2331
507,2021,4,10,Sant Martí,70,el Besòs i el Maresme,700.5,10.8,2332
508,2021,4,10,Sant Martí,71,Provençals del Poblenou,977.8,13.8,2333
509,2021,4,10,Sant Martí,72,Sant Martí de Provençals,826.2,11.7,2334


In [36]:
res.to_csv('rent_data.csv')

In [41]:
res.set_index('id').T.to_json('rent_data.json')

In [29]:
res.apply(lambda x: print(x.to_json()), axis=1)

{"Year":2014,"Trimestre":1,"Codi_Districte":1,"Nom_Districte":"Ciutat Vella","Codi_Barri":1,"Nom_Barri":"el Raval","Preu":589.55,"PreuM2":10.76,"id":0}
{"Year":2014,"Trimestre":1,"Codi_Districte":1,"Nom_Districte":"Ciutat Vella","Codi_Barri":2,"Nom_Barri":"el Barri G\u00c3\u00b2tic","Preu":712.79,"PreuM2":10.58,"id":1}
{"Year":2014,"Trimestre":1,"Codi_Districte":1,"Nom_Districte":"Ciutat Vella","Codi_Barri":3,"Nom_Barri":"la Barceloneta","Preu":540.71,"PreuM2":14.4,"id":2}
{"Year":2014,"Trimestre":1,"Codi_Districte":1,"Nom_Districte":"Ciutat Vella","Codi_Barri":4,"Nom_Barri":"Sant Pere, Santa Caterina i la Ribera","Preu":673.44,"PreuM2":11.01,"id":3}
{"Year":2014,"Trimestre":1,"Codi_Districte":2,"Nom_Districte":"Eixample","Codi_Barri":5,"Nom_Barri":"el Fort Pienc","Preu":736.09,"PreuM2":10.42,"id":4}
{"Year":2014,"Trimestre":1,"Codi_Districte":2,"Nom_Districte":"Eixample","Codi_Barri":6,"Nom_Barri":"la Sagrada Fam\u00c3\u00adlia","Preu":673.37,"PreuM2":10.55,"id":5}
{"Year":2014,"Trime

{"Year":2020,"Trimestre":1,"Codi_Districte":8,"Nom_Districte":"Nou Barris","Codi_Barri":44,"Nom_Barri":"Vilapicina i la Torre Llobeta","Preu":789.9,"PreuM2":12.3,"id":1795}
{"Year":2020,"Trimestre":1,"Codi_Districte":8,"Nom_Districte":"Nou Barris","Codi_Barri":45,"Nom_Barri":"Porta","Preu":690.4,"PreuM2":11.4,"id":1796}
{"Year":2020,"Trimestre":1,"Codi_Districte":8,"Nom_Districte":"Nou Barris","Codi_Barri":46,"Nom_Barri":"el Tur\u00f3 de la Peira","Preu":686.4,"PreuM2":11.4,"id":1797}
{"Year":2020,"Trimestre":1,"Codi_Districte":8,"Nom_Districte":"Nou Barris","Codi_Barri":47,"Nom_Barri":"Can Peguera","Preu":null,"PreuM2":null,"id":1798}
{"Year":2020,"Trimestre":1,"Codi_Districte":8,"Nom_Districte":"Nou Barris","Codi_Barri":48,"Nom_Barri":"la Guineueta","Preu":735.4,"PreuM2":10.2,"id":1799}
{"Year":2020,"Trimestre":1,"Codi_Districte":8,"Nom_Districte":"Nou Barris","Codi_Barri":49,"Nom_Barri":"Canyelles","Preu":778.0,"PreuM2":10.1,"id":1800}
{"Year":2020,"Trimestre":1,"Codi_Districte":8,"

0      None
1      None
2      None
3      None
4      None
       ... 
506    None
507    None
508    None
509    None
510    None
Length: 2336, dtype: object